In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam

# Constants
IMG_WIDTH, IMG_HEIGHT = 299, 299
BATCH_SIZE = 32
NUM_CLASSES = 4

# Load and preprocess data
data_generator = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = data_generator.flow_from_directory(
    'D:/DATASET/CNN/steatosis/train',
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

validation_generator = data_generator.flow_from_directory(
    'D:/DATASET/CNN/steatosis/val',
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Data augmentation
augmented_data_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Resampling using SMOTE for class 3
class3_images = np.load('path/to/class3_images.npy')
class3_labels = np.load('path/to/class3_labels.npy')

smote = SMOTE(random_state=42)
class3_images_resampled, class3_labels_resampled = smote.fit_resample(class3_images, class3_labels)

# Balance out class 3
class3_resampled_path = 'path/to/class3_resampled/'
for i, image in enumerate(class3_images_resampled):
    plt.imsave(class3_resampled_path + f'image_{i}.png', image)

# Model creation
base_model = ResNet50(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Training
history = model.fit(
    augmented_data_generator.flow(train_generator),
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

# Save the model
model.save('path/to/saved_model.h5')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.applications import ResNet50
from keras.models import Model
from skimage.transform import resize
from keras.applications.resnet50 import preprocess_input, decode_predictions

# Load pre-trained ResNet50 model
model = ResNet50(weights='imagenet')
model = Model(inputs=model.input, outputs=model.layers[-2].output)

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array, img

def generate_heatmap(img_array, model):
    preds = model.predict(img_array)
    class_idx = np.argmax(preds[0])
    class_output = model.output[:, class_idx]
    last_conv_layer = model.get_layer('conv5_block3_out')
    grads = K.gradients(class_output, last_conv_layer.output)[0]
    pooled_grads = K.mean(grads, axis=(0, 1, 2))
    iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])
    pooled_grads_value, conv_layer_output_value = iterate([img_array])
    for i in range(512):
        conv_layer_output_value[:, :, i] *= pooled_grads_value[i]
    heatmap = np.mean(conv_layer_output_value, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    return heatmap

def plot_heatmap(heatmap, img):
    heatmap = resize(heatmap, (img.shape[0], img.shape[1]))
    plt.imshow(img)
    plt.imshow(heatmap, alpha=0.5, cmap='jet')
    plt.show()

def predict_and_heatmap(img_path):
    img_array, img = preprocess_image(img_path)
    heatmap = generate_heatmap(img_array, model)
    plot_heatmap(heatmap, img)

# Example usage
image_path = 'path/to/your/image.png'
predict_and_heatmap(image_path)
